In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import pandas as pd

file_path = "/content/drive/My Drive/IS_project/ramen.csv"
df = pd.read_csv(file_path)

print(df.head())


   Review #           Brand  \
0      2580       New Touch   
1      2579        Just Way   
2      2578          Nissin   
3      2577         Wei Lih   
4      2576  Ching's Secret   

                                             Variety Style Country Stars  
0                          T's Restaurant Tantanmen    Cup   Japan  3.75  
1  Noodles Spicy Hot Sesame Spicy Hot Sesame Guan...  Pack  Taiwan     1  
2                      Cup Noodles Chicken Vegetable   Cup     USA  2.25  
3                      GGE Ramen Snack Tomato Flavor  Pack  Taiwan  2.75  
4                                    Singapore Curry  Pack   India  3.75  


In [32]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error
import os
import joblib

In [33]:
file_path = "/content/drive/My Drive/IS_project/ramen.csv"
ramen_df = pd.read_csv(file_path)

In [34]:
# แปลง 'Stars' เป็นตัวเลข และแทน NaN ด้วยค่าเฉลี่ย
ramen_df['Stars'] = pd.to_numeric(ramen_df['Stars'], errors='coerce')
ramen_df['Stars'].fillna(ramen_df['Stars'].mean(), inplace=True)

# ลบค่าที่เป็น NaN ออก
ramen_df.dropna(inplace=True)


<ipython-input-34-c00284d07647>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ramen_df['Stars'].fillna(ramen_df['Stars'].mean(), inplace=True)


In [35]:
# Label Encoding
label_encoders = {}
for col in ['Brand', 'Variety', 'Style', 'Country']:
    le = LabelEncoder()
    ramen_df[col] = le.fit_transform(ramen_df[col])
    label_encoders[col] = le

# แยก Features กับ Target
X = ramen_df[['Brand', 'Variety', 'Style', 'Country']]
y = ramen_df['Stars']

# แบ่งข้อมูลเป็น Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#  StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),  # เพิ่มขนาด Layer
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1)
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss=tf.keras.losses.Huber(),
              metrics=['mae'])

# Early stopping เพื่อป้องกัน Overfitting
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train
model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[early_stop])

#  บันทึกโมเดล
os.makedirs("models", exist_ok=True)
joblib.dump(scaler, "models/ramen_scaler.pkl")
joblib.dump(label_encoders, "models/ramen_label_encoders.pkl")
model.save("models/ramen_model.keras")


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 2.0119 - mae: 2.4806 - val_loss: 0.6736 - val_mae: 1.0753
Epoch 2/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6952 - mae: 1.0979 - val_loss: 0.5570 - val_mae: 0.9426
Epoch 3/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5814 - mae: 0.9653 - val_loss: 0.4729 - val_mae: 0.8372
Epoch 4/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5778 - mae: 0.9639 - val_loss: 0.4352 - val_mae: 0.7949
Epoch 5/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5090 - mae: 0.8784 - val_loss: 0.4033 - val_mae: 0.7570
Epoch 6/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4837 - mae: 0.8511 - val_loss: 0.3818 - val_mae: 0.7289
Epoch 7/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4530 - mae: 0.8186 - val_loss: 0.3743 - val_mae: 0.7210
Epoch 8/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4719 - mae: 0.8387 - val_loss: 0.3638 - val_mae: 0.7013
Epoch 9/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4246 - mae: 0.7

In [ ]:
from google.colab import files

files.download("models/ramen_scaler.pkl")
files.download("models/ramen_label_encoders.pkl")
files.download("models/ramen_model.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>